In [1]:
from google.colab import files
uploaded = files.upload()


Saving diabetic_data.csv to diabetic_data.csv


In [4]:
import pandas as pd

# Load the data
df = pd.read_csv("/content/diabetic_data.csv")

In [5]:
# Quick inspection
print("Rows, Columns:", df.shape)
print(df.head())
print(df.info())
print(df.isna().sum().sort_values(ascending=False))

Rows, Columns: (101766, 50)
   encounter_id  patient_nbr             race  gender      age weight  \
0       2278392      8222157        Caucasian  Female   [0-10)      ?   
1        149190     55629189        Caucasian  Female  [10-20)      ?   
2         64410     86047875  AfricanAmerican  Female  [20-30)      ?   
3        500364     82442376        Caucasian    Male  [30-40)      ?   
4         16680     42519267        Caucasian    Male  [40-50)      ?   

   admission_type_id  discharge_disposition_id  admission_source_id  \
0                  6                        25                    1   
1                  1                         1                    7   
2                  1                         1                    7   
3                  1                         1                    7   
4                  1                         1                    7   

   time_in_hospital  ... citoglipton insulin  glyburide-metformin  \
0                 1  ...          No 

In [6]:
# Count '?' values in each column
missing_question = (df == "?").sum().sort_values(ascending=False)

missing_question.head(10)

,0
weight,98569
medical_specialty,49949
payer_code,40256
race,2273
diag_3,1423
diag_2,358
diag_1,21
admission_type_id,0
patient_nbr,0
encounter_id,0


In [7]:
# Replace '?' with NaN
df.replace("?", pd.NA, inplace=True)

# Drop high-missing columns
columns_to_drop = ["weight", "medical_specialty", "payer_code"]
df.drop(columns=columns_to_drop, inplace=True)

df.shape


(101766, 47)

In [8]:
df["readmitted"].value_counts()


,count
readmitted,
NO,54864
>30,35545
<30,11357


In [9]:
df["readmitted_flag"] = df["readmitted"].apply(
    lambda x: 1 if x == "<30" else 0
)

df["readmitted_flag"].value_counts()


,count
readmitted_flag,
0,90409
1,11357


In [10]:
df["diag_1"].value_counts().head(10)


,count
diag_1,
428,6862
414,6581
786,4016
410,3614
486,3508
427,2766
491,2275
715,2151
682,2042


In [11]:
def categorize_diag(code):
    try:
        code = float(code)
    except:
        return "Other"

    if 390 <= code <= 459:
        return "Circulatory"
    elif 460 <= code <= 519:
        return "Respiratory"
    elif 520 <= code <= 579:
        return "Digestive"
    elif code == 250:
        return "Diabetes"
    elif 800 <= code <= 999:
        return "Injury"
    else:
        return "Other"

df["diag_1_category"] = df["diag_1"].apply(categorize_diag)

df["diag_1_category"].value_counts()


,count
diag_1_category,
Other,44606
Circulatory,30336
Respiratory,10407
Digestive,9208
Injury,6974
Diabetes,235


In [12]:
readmission_by_diag = df.groupby("diag_1_category")["readmitted_flag"].mean().sort_values(ascending=False)

readmission_by_diag


,readmitted_flag
diag_1_category,
Injury,0.122455
Circulatory,0.114517
Other,0.110635
Respiratory,0.106851
Digestive,0.104909
Diabetes,0.068085


In [13]:
df["diag_2_category"] = df["diag_2"].apply(categorize_diag)
df["diag_3_category"] = df["diag_3"].apply(categorize_diag)


In [14]:
df["circulatory_flag"] = (
    (df["diag_1_category"] == "Circulatory") |
    (df["diag_2_category"] == "Circulatory") |
    (df["diag_3_category"] == "Circulatory")
).astype(int)

df["circulatory_flag"].value_counts()


,count
circulatory_flag,
1,58672
0,43094


In [15]:
df["injury_flag"] = (
    (df["diag_1_category"] == "Injury") |
    (df["diag_2_category"] == "Injury") |
    (df["diag_3_category"] == "Injury")
).astype(int)

df["injury_flag"].value_counts()


,count
injury_flag,
0,91797
1,9969


In [16]:
## Readmission by Circulatory Flag
df.groupby("circulatory_flag")["readmitted_flag"].mean()


,readmitted_flag
circulatory_flag,
0,0.111268
1,0.111842


In [17]:
## Readmission by Injury Flag
df.groupby("injury_flag")["readmitted_flag"].mean()


,readmitted_flag
injury_flag,
0,0.110625
1,0.120574


In [18]:
## Previous Admissions & Visit History
df["total_visits"] = (
    df["number_outpatient"] +
    df["number_emergency"] +
    df["number_inpatient"]
)

df["total_visits"].describe()


,total_visits
count,101766.000000
mean,1.202759
std,2.291781
min,0.000000
25%,0.000000
50%,0.000000
75%,2.000000
max,80.000000


In [19]:
df.groupby("total_visits")["readmitted_flag"].mean().sort_index().head(15)


,readmitted_flag
total_visits,
0,0.081751
1,0.115641
2,0.146194
3,0.152778
4,0.171148
5,0.183694
6,0.232292
7,0.226959
8,0.245487


In [20]:
def visit_risk_tier(x):
    if x == 0:
        return "Low"
    elif 1 <= x <= 2:
        return "Medium"
    elif 3 <= x <= 5:
        return "High"
    else:
        return "Very_High"

df["visit_risk_tier"] = df["total_visits"].apply(visit_risk_tier)

df.groupby("visit_risk_tier")["readmitted_flag"].mean().sort_values()


,readmitted_flag
visit_risk_tier,
Low,0.081751
Medium,0.125887
High,0.163944
Very_High,0.255141


In [21]:
df["time_in_hospital"].describe()


,time_in_hospital
count,101766.000000
mean,4.395987
std,2.985108
min,1.000000
25%,2.000000
50%,4.000000
75%,6.000000
max,14.000000


In [22]:
df.groupby("time_in_hospital")["readmitted_flag"].mean().head(15)


,readmitted_flag
time_in_hospital,
1,0.081785
2,0.099396
3,0.106668
4,0.118070
5,0.120309
6,0.125879
7,0.128350
8,0.142337
9,0.137242


In [23]:
## Create LOS Risk Tier
def los_tier(x):
    if x <= 2:
        return "Short"
    elif 3 <= x <= 5:
        return "Medium"
    elif 6 <= x <= 8:
        return "Long"
    else:
        return "Extended"

df["los_tier"] = df["time_in_hospital"].apply(los_tier)

df.groupby("los_tier")["readmitted_flag"].mean().sort_values()


,readmitted_flag
los_tier,
Short,0.091435
Medium,0.113744
Extended,0.130287
Long,0.130755


In [24]:
df["number_diagnoses"].describe()


,number_diagnoses
count,101766.000000
mean,7.422607
std,1.933600
min,1.000000
25%,6.000000
50%,8.000000
75%,9.000000
max,16.000000


In [25]:
df.groupby("number_diagnoses")["readmitted_flag"].mean().head(15)


,readmitted_flag
number_diagnoses,
1,0.059361
2,0.060606
3,0.073721
4,0.082536
5,0.091547
6,0.104124
7,0.107669
8,0.118124
9,0.123802


In [26]:
def diag_tier(x):
    if x <= 3:
        return "Low"
    elif 4 <= x <= 7:
        return "Medium"
    elif 8 <= x <= 10:
        return "High"
    else:
        return "Very_High"

df["diagnosis_tier"] = df["number_diagnoses"].apply(diag_tier)

df.groupby("diagnosis_tier")["readmitted_flag"].mean().sort_values()


,readmitted_flag
diagnosis_tier,
Low,0.069659
Medium,0.098095
High,0.122814
Very_High,0.142857


## **Modeling**

In [27]:
## feature matrix.
model_features = [
    "visit_risk_tier",
    "los_tier",
    "diagnosis_tier",
    "circulatory_flag",
    "injury_flag",
    "time_in_hospital",
    "total_visits",
    "number_diagnoses"
]

df_model = df[model_features + ["readmitted_flag"]].copy()

df_model.head()


,visit_risk_tier,los_tier,diagnosis_tier,circulatory_flag,injury_flag,time_in_hospital,total_visits,number_diagnoses,readmitted_flag
0,Low,Short,Low,0,0,1,0,1,0
1,Low,Medium,High,0,0,3,0,9,0
2,High,Short,Medium,0,0,2,3,6,0
3,Low,Short,Medium,1,0,2,0,7,0
4,Low,Short,Medium,0,0,1,0,5,0


In [28]:
df_encoded = pd.get_dummies(
    df_model,
    columns=["visit_risk_tier", "los_tier", "diagnosis_tier"],
    drop_first=True
)

df_encoded.head()


,circulatory_flag,injury_flag,time_in_hospital,total_visits,number_diagnoses,readmitted_flag,visit_risk_tier_Low,visit_risk_tier_Medium,visit_risk_tier_Very_High,los_tier_Long,los_tier_Medium,los_tier_Short,diagnosis_tier_Low,diagnosis_tier_Medium,diagnosis_tier_Very_High
0,0,0,1,0,1,0,True,False,False,False,False,True,True,False,False
1,0,0,3,0,9,0,True,False,False,False,True,False,False,False,False
2,0,0,2,3,6,0,False,False,False,False,False,True,False,True,False
3,1,0,2,0,7,0,True,False,False,False,False,True,False,True,False
4,0,0,1,0,5,0,True,False,False,False,False,True,False,True,False


In [29]:
df_encoded = df_encoded.astype(int)
df_encoded.head()


,circulatory_flag,injury_flag,time_in_hospital,total_visits,number_diagnoses,readmitted_flag,visit_risk_tier_Low,visit_risk_tier_Medium,visit_risk_tier_Very_High,los_tier_Long,los_tier_Medium,los_tier_Short,diagnosis_tier_Low,diagnosis_tier_Medium,diagnosis_tier_Very_High
0,0,0,1,0,1,0,1,0,0,0,0,1,1,0,0
1,0,0,3,0,9,0,1,0,0,0,1,0,0,0,0
2,0,0,2,3,6,0,0,0,0,0,0,1,0,1,0
3,1,0,2,0,7,0,1,0,0,0,0,1,0,1,0
4,0,0,1,0,5,0,1,0,0,0,0,1,0,1,0


In [34]:
## Separate X and y
X = df_encoded.drop("readmitted_flag", axis=1)
y = df_encoded["readmitted_flag"]


In [35]:
## Train/Test Split
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

X_train.shape, X_test.shape


((81412, 14), (20354, 14))

In [36]:
## Train Logistic Regression
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)


LogisticRegression(max_iter=1000)

In [37]:
## Evaluate Model
from sklearn.metrics import classification_report, roc_auc_score

y_pred = model.predict(X_test)
y_prob = model.predict_proba(X_test)[:, 1]

print(classification_report(y_test, y_pred))
print("ROC-AUC:", roc_auc_score(y_test, y_prob))


              precision    recall  f1-score   support

           0       0.89      1.00      0.94     18083
           1       0.50      0.00      0.00      2271

    accuracy                           0.89     20354
   macro avg       0.69      0.50      0.47     20354
weighted avg       0.85      0.89      0.84     20354

ROC-AUC: 0.6236057094040146


In [38]:
model = LogisticRegression(max_iter=1000, class_weight="balanced")
model.fit(X_train, y_train)


LogisticRegression(class_weight='balanced', max_iter=1000)

In [39]:
y_pred = model.predict(X_test)
y_prob = model.predict_proba(X_test)[:, 1]

print(classification_report(y_test, y_pred))
print("ROC-AUC:", roc_auc_score(y_test, y_prob))


              precision    recall  f1-score   support

           0       0.92      0.63      0.75     18083
           1       0.16      0.54      0.24      2271

    accuracy                           0.62     20354
   macro avg       0.54      0.59      0.50     20354
weighted avg       0.83      0.62      0.69     20354

ROC-AUC: 0.6240090065640618


In [40]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(
    n_estimators=200,
    class_weight="balanced",
    random_state=42
)

rf.fit(X_train, y_train)

y_pred = rf.predict(X_test)
y_prob = rf.predict_proba(X_test)[:, 1]

print(classification_report(y_test, y_pred))
print("ROC-AUC:", roc_auc_score(y_test, y_prob))


              precision    recall  f1-score   support

           0       0.91      0.69      0.78     18083
           1       0.15      0.42      0.22      2271

    accuracy                           0.66     20354
   macro avg       0.53      0.56      0.50     20354
weighted avg       0.82      0.66      0.72     20354

ROC-AUC: 0.5670578931587852


In [42]:
from google.colab import files
files.download("hospital_readmission_cleaned.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>